In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import seaborn as sns
#from statsmodels.graphics.mosaicplot import mosaic
import io
import requests

%matplotlib inline


import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import log_loss

pd.set_option('display.max_columns', None)

In [23]:
df2 = pd.read_csv('dataset_v2').set_index('building_id').drop(columns=['damage_grade', 'Unnamed: 0'])
df1 = pd.read_csv('dataset_v1').set_index('building_id')
train_labels = pd.read_csv('train_labels.csv').set_index('building_id')

In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 53 columns):
 #   Column                                  Non-Null Count   Dtype
---  ------                                  --------------   -----
 0   geo_level_1_id                          260601 non-null  int64
 1   geo_level_2_id                          260601 non-null  int64
 2   geo_level_3_id                          260601 non-null  int64
 3   count_floors_pre_eq                     260601 non-null  int64
 4   age                                     260601 non-null  int64
 5   area_percentage                         260601 non-null  int64
 6   height_percentage                       260601 non-null  int64
 7   has_superstructure_adobe_mud            260601 non-null  bool 
 8   has_superstructure_mud_mortar_stone     260601 non-null  bool 
 9   has_superstructure_stone_flag           260601 non-null  bool 
 10  has_superstructure_cement_mortar_stone  260601 non-null  bool 


In [26]:
# df2 es mi set de entrenamiento
train_labels.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


In [27]:
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostClassifier

In [34]:
x, y = df2, train_labels

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2048)

In [69]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208480 entries, 285985 to 370619
Data columns (total 36 columns):
 #   Column                                  Non-Null Count   Dtype
---  ------                                  --------------   -----
 0   geo_level_1_id                          208480 non-null  int64
 1   geo_level_2_id                          208480 non-null  int64
 2   geo_level_3_id                          208480 non-null  int64
 3   age                                     208480 non-null  int64
 4   area_percentage                         208480 non-null  int64
 5   height_percentage                       208480 non-null  int64
 6   has_superstructure_adobe_mud            208480 non-null  bool 
 7   has_superstructure_mud_mortar_stone     208480 non-null  bool 
 8   has_superstructure_stone_flag           208480 non-null  bool 
 9   has_superstructure_cement_mortar_stone  208480 non-null  bool 
 10  has_superstructure_mud_mortar_brick     208480 non-null  bool 


# RandomForest


In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
clfRF = RandomForestClassifier(min_samples_leaf=5, n_estimators=50, max_features="log2")

In [39]:
clfRF.fit(x_train, y_train)

/home/fscaccheri/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(max_features='log2', min_samples_leaf=5, n_estimators=50)

In [40]:
y_pred = clfRF.predict(x_test)
accuracy_score(y_test, y_pred)

0.6938853820916713

# XGBoost

In [41]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

In [73]:
arr_x = np.array(x)
arr_y = np.array(y)

In [74]:
from sklearn.model_selection import GridSearchCV

In [105]:
param_grid = {
    'learning_rate': [0.03, 0.1],
    'n_estimators': [100, 120],
    'reg_lambda': [1e-05, 3e-05],
    'reg_alpha': [0.1, 0.5],
    'colsample_bynode': [0.3, 0.8],
    'max_depth': [10, 15]
}

xgb_model = xgb.XGBRFClassifier(
    random_state=40,
    max_depth=10,
    colsample_bynode=0.8
)

gs = GridSearchCV(xgb_model, param_grid, cv=5, verbose=3)

In [106]:
gs.fit(x_train, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


/home/fscaccheri/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/fscaccheri/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:41:19] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5] END colsample_bynode=0.3, learning_rate=0.03, max_depth=10, n_estimators=100, reg_alpha=0.1, reg_lambda=1e-05;, score=0.674 total time=  15.1s


/home/fscaccheri/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/fscaccheri/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:41:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5] END colsample_bynode=0.3, learning_rate=0.03, max_depth=10, n_estimators=100, reg_alpha=0.1, reg_lambda=1e-05;, score=0.676 total time=  16.0s


...

/home/fscaccheri/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/fscaccheri/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[19:35:10] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 5/5] END colsample_bynode=0.8, learning_rate=0.1, max_depth=15, n_estimators=120, reg_alpha=0.5, reg_lambda=3e-05;, score=0.729 total time= 1.2min


/home/fscaccheri/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/fscaccheri/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[19:36:21] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBRFClassifier(base_score=None, booster=None,
                                       colsample_bylevel=None,
                                       colsample_bytree=None, gamma=None,
                                       gpu_id=None, importance_type='gain',
                                       interaction_constraints=None,
                                       max_delta_step=None, max_depth=10,
                                       min_child_weight=None, missing=nan,
                                       monotone_constraints=None,
                                       n_estimators=100, n_jobs=None,
                                       num_parallel_tree=None,
                                       objective='binary:logistic',
                                       random_state=40, reg_alpha=None,
                                       scale_pos_weight=None, tree_method=None,
                                       validate_parameters=No

In [107]:
gs.best_params_

{'colsample_bynode': 0.8,
 'learning_rate': 0.1,
 'max_depth': 15,
 'n_estimators': 100,
 'reg_alpha': 0.1,
 'reg_lambda': 3e-05}

In [123]:
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(arr_y, arr_y):
    xgb_model = xgb.XGBRFClassifier(
        random_state=40,
        colsample_bynode=0.8,
        learning_rate=0.1,
        max_depth=15,
        n_estimators= 100,
        reg_alpha=0.1,
        reg_lambda=3e-05
    )
    xgb_model.fit(arr_x[train_index], arr_y[train_index])
    
'''
parametros del intento anterior
random_state=40,
colsample_bynode=0.8,
learning_rate=0.3,
max_depth=10,
n_estimators= 100,
reg_alpha=0.5,
reg_lambda=1e-05
'''

/home/fscaccheri/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/fscaccheri/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[21:32:21] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:32:50] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:33:18] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


'\nparametros del intento anterior\nrandom_state=40,\ncolsample_bynode=0.8,\nlearning_rate=0.3,\nmax_depth=10,\nn_estimators= 100,\nreg_alpha=0.5,\nreg_lambda=1e-05\n\nlos nuevos:\nrandom_state=40,\ncolsample_bynode=0.8,\nlearning_rate=0.1,\nmax_depth=15,\nn_estimators= 100,\nreg_alpha=0.1,\nreg_lambda=3e-05\n'

In [124]:
score = xgb_model.score(x, y)
print("accuracy=%.2f%%" % ( score * 100))

accuracy=70.12%


# Catboost

In [88]:
cbm = CatBoostClassifier(
    learning_rate=0.3,
    depth=5,
    loss_function='MultiClass'
)

#cbm.fit(x, y)

In [49]:
from sklearn.metrics import f1_score

preds = cbm.predict(x)
f1_score(y, preds, average='micro')

0.7482051104945875

In [50]:
cbm.score(x,y)

0.7482051104945875

# VotingClassifier

In [52]:
from sklearn.ensemble import VotingClassifier

In [125]:
eclf = VotingClassifier(estimators=[('rf', clfRF),('xgb', xgb_model), ('cb', cbm)], voting='soft')

In [111]:
eclf = eclf.fit(x_train, y_train)

/home/fscaccheri/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/fscaccheri/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:09:05] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0:	learn: 0.9685935	total: 50.3ms	remaining: 50.2s
1:	learn: 0.8864004	total: 90.4ms	remaining: 45.1s
2:	learn: 0.8423944	total: 132ms	remaining: 43.8s
3:	learn: 0.8130961	total: 165ms	remaining: 41s
4:	learn: 0.7898322	total: 205ms	remaining: 40.8s
5:	learn: 0.7740178	total: 243ms	remaining: 40.3s
6:	learn: 0.7647921	total: 292ms	remaining: 41.4s
7:	learn: 0.7548027	total: 336ms	remaining: 41.6s
8:	learn: 0.7444708	total: 389ms	remaining: 42.8s
9:	learn: 0.7386741	total: 428ms	remaining: 42.4s
10:	learn: 0.7325371	total: 479ms	remaining: 43s
11:	learn: 0.7280504	total: 516ms	remaining: 42.4s
12:	learn: 0.7254275	total: 552ms	remaining: 41.9s
13:	learn: 0.7223992	total: 585ms	remaining: 41.2s
14:	learn: 0.7181275	total: 628ms	remaining: 

159:	learn: 0.6434395	total: 6.14s	remaining: 32.2s
160:	learn: 0.6432942	total: 6.18s	remaining: 32.2s
161:	learn: 0.6431832	total: 6.21s	remaining: 32.1s
162:	learn: 0.6430686	total: 6.25s	remaining: 32.1s
163:	learn: 0.6429243	total: 6.3s	remaining: 32.1s
164:	learn: 0.6427332	total: 6.35s	remaining: 32.1s
165:	learn: 0.6423652	total: 6.38s	remaining: 32.1s
166:	learn: 0.6420604	total: 6.43s	remaining: 32.1s
167:	learn: 0.6419183	total: 6.46s	remaining: 32s
168:	learn: 0.6418094	total: 6.49s	remaining: 31.9s
169:	learn: 0.6416834	total: 6.52s	remaining: 31.8s
170:	learn: 0.6416186	total: 6.56s	remaining: 31.8s
171:	learn: 0.6413800	total: 6.6s	remaining: 31.8s
172:	learn: 0.6411600	total: 6.64s	remaining: 31.8s
173:	learn: 0.6410462	total: 6.68s	remaining: 31.7s
174:	learn: 0.6408884	total: 6.72s	remaining: 31.7s
175:	learn: 0.6407370	total: 6.76s	remaining: 31.7s
176:	learn: 0.6404449	total: 6.79s	remaining: 31.6s
177:	learn: 0.6402203	total: 6.83s	remaining: 31.6s
178:	learn: 0.64

318:	learn: 0.6197021	total: 12.3s	remaining: 26.3s
319:	learn: 0.6195904	total: 12.4s	remaining: 26.3s
320:	learn: 0.6195279	total: 12.4s	remaining: 26.3s
321:	learn: 0.6194127	total: 12.5s	remaining: 26.2s
322:	learn: 0.6190565	total: 12.5s	remaining: 26.2s
323:	learn: 0.6187990	total: 12.5s	remaining: 26.2s
324:	learn: 0.6187495	total: 12.6s	remaining: 26.1s
325:	learn: 0.6185882	total: 12.6s	remaining: 26.1s
326:	learn: 0.6184869	total: 12.7s	remaining: 26s
327:	learn: 0.6184043	total: 12.7s	remaining: 26s
328:	learn: 0.6182318	total: 12.7s	remaining: 26s
329:	learn: 0.6181935	total: 12.8s	remaining: 25.9s
330:	learn: 0.6178536	total: 12.8s	remaining: 25.9s
331:	learn: 0.6178066	total: 12.8s	remaining: 25.8s
332:	learn: 0.6177327	total: 12.9s	remaining: 25.8s
333:	learn: 0.6176949	total: 12.9s	remaining: 25.7s
334:	learn: 0.6176297	total: 12.9s	remaining: 25.7s
335:	learn: 0.6175384	total: 13s	remaining: 25.6s
336:	learn: 0.6173937	total: 13s	remaining: 25.6s
337:	learn: 0.6172807	

479:	learn: 0.6028773	total: 18.6s	remaining: 20.1s
480:	learn: 0.6027915	total: 18.6s	remaining: 20.1s
481:	learn: 0.6027092	total: 18.6s	remaining: 20s
482:	learn: 0.6025877	total: 18.7s	remaining: 20s
483:	learn: 0.6024953	total: 18.7s	remaining: 19.9s
484:	learn: 0.6024207	total: 18.7s	remaining: 19.9s
485:	learn: 0.6023124	total: 18.8s	remaining: 19.9s
486:	learn: 0.6022250	total: 18.8s	remaining: 19.8s
487:	learn: 0.6021522	total: 18.9s	remaining: 19.8s
488:	learn: 0.6020834	total: 18.9s	remaining: 19.8s
489:	learn: 0.6020106	total: 18.9s	remaining: 19.7s
490:	learn: 0.6019536	total: 19s	remaining: 19.7s
491:	learn: 0.6019070	total: 19s	remaining: 19.6s
492:	learn: 0.6018492	total: 19s	remaining: 19.6s
493:	learn: 0.6017797	total: 19.1s	remaining: 19.5s
494:	learn: 0.6017109	total: 19.1s	remaining: 19.5s
495:	learn: 0.6016938	total: 19.2s	remaining: 19.5s
496:	learn: 0.6016200	total: 19.2s	remaining: 19.4s
497:	learn: 0.6015736	total: 19.2s	remaining: 19.4s
498:	learn: 0.6014718	

640:	learn: 0.5909504	total: 24.7s	remaining: 13.8s
641:	learn: 0.5909188	total: 24.7s	remaining: 13.8s
642:	learn: 0.5908021	total: 24.7s	remaining: 13.7s
643:	learn: 0.5907522	total: 24.8s	remaining: 13.7s
644:	learn: 0.5906902	total: 24.8s	remaining: 13.7s
645:	learn: 0.5906003	total: 24.8s	remaining: 13.6s
646:	learn: 0.5905610	total: 24.9s	remaining: 13.6s
647:	learn: 0.5904564	total: 24.9s	remaining: 13.5s
648:	learn: 0.5903267	total: 25s	remaining: 13.5s
649:	learn: 0.5902895	total: 25s	remaining: 13.5s
650:	learn: 0.5902691	total: 25s	remaining: 13.4s
651:	learn: 0.5902281	total: 25.1s	remaining: 13.4s
652:	learn: 0.5901778	total: 25.1s	remaining: 13.3s
653:	learn: 0.5901214	total: 25.1s	remaining: 13.3s
654:	learn: 0.5900680	total: 25.2s	remaining: 13.3s
655:	learn: 0.5900340	total: 25.2s	remaining: 13.2s
656:	learn: 0.5899764	total: 25.2s	remaining: 13.2s
657:	learn: 0.5899266	total: 25.3s	remaining: 13.1s
658:	learn: 0.5898347	total: 25.3s	remaining: 13.1s
659:	learn: 0.5898

800:	learn: 0.5816818	total: 30.6s	remaining: 7.61s
801:	learn: 0.5816389	total: 30.7s	remaining: 7.57s
802:	learn: 0.5815951	total: 30.7s	remaining: 7.53s
803:	learn: 0.5815474	total: 30.7s	remaining: 7.5s
804:	learn: 0.5814629	total: 30.8s	remaining: 7.46s
805:	learn: 0.5814230	total: 30.8s	remaining: 7.42s
806:	learn: 0.5813705	total: 30.9s	remaining: 7.38s
807:	learn: 0.5813364	total: 30.9s	remaining: 7.34s
808:	learn: 0.5812852	total: 30.9s	remaining: 7.3s
809:	learn: 0.5812557	total: 31s	remaining: 7.27s
810:	learn: 0.5811783	total: 31s	remaining: 7.23s
811:	learn: 0.5811440	total: 31.1s	remaining: 7.19s
812:	learn: 0.5810899	total: 31.1s	remaining: 7.15s
813:	learn: 0.5810482	total: 31.1s	remaining: 7.11s
814:	learn: 0.5809377	total: 31.2s	remaining: 7.08s
815:	learn: 0.5809076	total: 31.2s	remaining: 7.04s
816:	learn: 0.5808576	total: 31.2s	remaining: 7s
817:	learn: 0.5808171	total: 31.3s	remaining: 6.96s
818:	learn: 0.5807761	total: 31.3s	remaining: 6.92s
819:	learn: 0.5807295

963:	learn: 0.5726966	total: 36.9s	remaining: 1.38s
964:	learn: 0.5726547	total: 36.9s	remaining: 1.34s
965:	learn: 0.5726272	total: 36.9s	remaining: 1.3s
966:	learn: 0.5725760	total: 37s	remaining: 1.26s
967:	learn: 0.5725422	total: 37s	remaining: 1.22s
968:	learn: 0.5725019	total: 37s	remaining: 1.18s
969:	learn: 0.5724157	total: 37.1s	remaining: 1.15s
970:	learn: 0.5723809	total: 37.1s	remaining: 1.11s
971:	learn: 0.5723395	total: 37.1s	remaining: 1.07s
972:	learn: 0.5722809	total: 37.2s	remaining: 1.03s
973:	learn: 0.5722491	total: 37.2s	remaining: 993ms
974:	learn: 0.5722181	total: 37.2s	remaining: 955ms
975:	learn: 0.5721392	total: 37.3s	remaining: 917ms
976:	learn: 0.5721117	total: 37.3s	remaining: 878ms
977:	learn: 0.5720800	total: 37.3s	remaining: 840ms
978:	learn: 0.5720144	total: 37.4s	remaining: 802ms
979:	learn: 0.5719824	total: 37.4s	remaining: 764ms
980:	learn: 0.5719448	total: 37.5s	remaining: 726ms
981:	learn: 0.5718810	total: 37.5s	remaining: 688ms
982:	learn: 0.57183

In [112]:
y_pred =eclf.predict(x_test)

In [113]:
accuracy_score(y_test, y_pred)

0.7251203929318317

In [126]:
# Ahora entrenamos con el set completo de datos
eclf = eclf.fit(x, y)

/home/fscaccheri/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/fscaccheri/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:34:28] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0:	learn: 0.9676455	total: 41.8ms	remaining: 41.7s
1:	learn: 0.8862174	total: 81.2ms	remaining: 40.5s
2:	learn: 0.8423965	total: 119ms	remaining: 39.4s
3:	learn: 0.8100768	total: 155ms	remaining: 38.5s
4:	learn: 0.7874932	total: 195ms	remaining: 38.7s
5:	learn: 0.7711127	total: 237ms	remaining: 39.2s
6:	learn: 0.7623528	total: 279ms	remaining: 39.6s
7:	learn: 0.7530237	total: 317ms	remaining: 39.4s
8:	learn: 0.7454301	total: 347ms	remaining: 38.3s
9:	learn: 0.7413175	total: 385ms	remaining: 38.1s
10:	learn: 0.7375856	total: 422ms	remaining: 37.9s
11:	learn: 0.7340904	total: 455ms	remaining: 37.5s
12:	learn: 0.7312689	total: 492ms	remaining: 37.4s
13:	learn: 0.7286741	total: 533ms	remaining: 37.5s
14:	learn: 0.7210874	total: 578ms	remaini

157:	learn: 0.6455715	total: 6.04s	remaining: 32.2s
158:	learn: 0.6452709	total: 6.08s	remaining: 32.1s
159:	learn: 0.6450223	total: 6.12s	remaining: 32.1s
160:	learn: 0.6447796	total: 6.15s	remaining: 32.1s
161:	learn: 0.6443494	total: 6.19s	remaining: 32s
162:	learn: 0.6441594	total: 6.23s	remaining: 32s
163:	learn: 0.6438900	total: 6.28s	remaining: 32s
164:	learn: 0.6438163	total: 6.32s	remaining: 32s
165:	learn: 0.6435119	total: 6.36s	remaining: 32s
166:	learn: 0.6434320	total: 6.39s	remaining: 31.9s
167:	learn: 0.6431005	total: 6.44s	remaining: 31.9s
168:	learn: 0.6429477	total: 6.48s	remaining: 31.9s
169:	learn: 0.6427608	total: 6.52s	remaining: 31.8s
170:	learn: 0.6425928	total: 6.56s	remaining: 31.8s
171:	learn: 0.6421934	total: 6.59s	remaining: 31.7s
172:	learn: 0.6419951	total: 6.63s	remaining: 31.7s
173:	learn: 0.6418485	total: 6.67s	remaining: 31.7s
174:	learn: 0.6416830	total: 6.7s	remaining: 31.6s
175:	learn: 0.6415153	total: 6.75s	remaining: 31.6s
176:	learn: 0.6412333	t

321:	learn: 0.6191210	total: 12.3s	remaining: 25.9s
322:	learn: 0.6190492	total: 12.3s	remaining: 25.8s
323:	learn: 0.6189871	total: 12.4s	remaining: 25.8s
324:	learn: 0.6189167	total: 12.4s	remaining: 25.8s
325:	learn: 0.6188892	total: 12.4s	remaining: 25.7s
326:	learn: 0.6188539	total: 12.5s	remaining: 25.7s
327:	learn: 0.6186767	total: 12.5s	remaining: 25.7s
328:	learn: 0.6186222	total: 12.6s	remaining: 25.6s
329:	learn: 0.6185239	total: 12.6s	remaining: 25.6s
330:	learn: 0.6184568	total: 12.6s	remaining: 25.5s
331:	learn: 0.6184158	total: 12.7s	remaining: 25.5s
332:	learn: 0.6182783	total: 12.7s	remaining: 25.5s
333:	learn: 0.6180804	total: 12.8s	remaining: 25.4s
334:	learn: 0.6179943	total: 12.8s	remaining: 25.4s
335:	learn: 0.6179127	total: 12.8s	remaining: 25.4s
336:	learn: 0.6177643	total: 12.9s	remaining: 25.3s
337:	learn: 0.6176510	total: 12.9s	remaining: 25.3s
338:	learn: 0.6175910	total: 12.9s	remaining: 25.2s
339:	learn: 0.6175341	total: 13s	remaining: 25.2s
340:	learn: 0.

485:	learn: 0.6044546	total: 18.5s	remaining: 19.5s
486:	learn: 0.6044029	total: 18.5s	remaining: 19.5s
487:	learn: 0.6042530	total: 18.6s	remaining: 19.5s
488:	learn: 0.6039362	total: 18.6s	remaining: 19.4s
489:	learn: 0.6038720	total: 18.6s	remaining: 19.4s
490:	learn: 0.6037503	total: 18.7s	remaining: 19.4s
491:	learn: 0.6037207	total: 18.7s	remaining: 19.3s
492:	learn: 0.6036667	total: 18.8s	remaining: 19.3s
493:	learn: 0.6033505	total: 18.8s	remaining: 19.3s
494:	learn: 0.6033054	total: 18.8s	remaining: 19.2s
495:	learn: 0.6032597	total: 18.9s	remaining: 19.2s
496:	learn: 0.6031642	total: 18.9s	remaining: 19.1s
497:	learn: 0.6030468	total: 18.9s	remaining: 19.1s
498:	learn: 0.6029946	total: 19s	remaining: 19.1s
499:	learn: 0.6028695	total: 19s	remaining: 19s
500:	learn: 0.6028222	total: 19.1s	remaining: 19s
501:	learn: 0.6027421	total: 19.1s	remaining: 19s
502:	learn: 0.6026580	total: 19.1s	remaining: 18.9s
503:	learn: 0.6025922	total: 19.2s	remaining: 18.9s
504:	learn: 0.6024652	

645:	learn: 0.5930316	total: 24.5s	remaining: 13.4s
646:	learn: 0.5929669	total: 24.5s	remaining: 13.4s
647:	learn: 0.5929316	total: 24.6s	remaining: 13.3s
648:	learn: 0.5928126	total: 24.6s	remaining: 13.3s
649:	learn: 0.5927557	total: 24.6s	remaining: 13.3s
650:	learn: 0.5926925	total: 24.7s	remaining: 13.2s
651:	learn: 0.5926639	total: 24.7s	remaining: 13.2s
652:	learn: 0.5926373	total: 24.7s	remaining: 13.1s
653:	learn: 0.5925854	total: 24.8s	remaining: 13.1s
654:	learn: 0.5925509	total: 24.8s	remaining: 13.1s
655:	learn: 0.5925145	total: 24.8s	remaining: 13s
656:	learn: 0.5924377	total: 24.9s	remaining: 13s
657:	learn: 0.5924162	total: 24.9s	remaining: 12.9s
658:	learn: 0.5923557	total: 24.9s	remaining: 12.9s
659:	learn: 0.5922452	total: 25s	remaining: 12.9s
660:	learn: 0.5921634	total: 25s	remaining: 12.8s
661:	learn: 0.5921234	total: 25.1s	remaining: 12.8s
662:	learn: 0.5920377	total: 25.1s	remaining: 12.8s
663:	learn: 0.5919806	total: 25.2s	remaining: 12.7s
664:	learn: 0.591901

809:	learn: 0.5835276	total: 30.7s	remaining: 7.2s
810:	learn: 0.5835024	total: 30.7s	remaining: 7.17s
811:	learn: 0.5834605	total: 30.8s	remaining: 7.13s
812:	learn: 0.5834189	total: 30.8s	remaining: 7.09s
813:	learn: 0.5833787	total: 30.8s	remaining: 7.05s
814:	learn: 0.5833397	total: 30.9s	remaining: 7.01s
815:	learn: 0.5832852	total: 30.9s	remaining: 6.97s
816:	learn: 0.5832382	total: 31s	remaining: 6.93s
817:	learn: 0.5832020	total: 31s	remaining: 6.9s
818:	learn: 0.5831740	total: 31s	remaining: 6.86s
819:	learn: 0.5830776	total: 31.1s	remaining: 6.82s
820:	learn: 0.5829929	total: 31.1s	remaining: 6.78s
821:	learn: 0.5829424	total: 31.1s	remaining: 6.74s
822:	learn: 0.5828812	total: 31.2s	remaining: 6.71s
823:	learn: 0.5828338	total: 31.2s	remaining: 6.67s
824:	learn: 0.5827832	total: 31.3s	remaining: 6.63s
825:	learn: 0.5827347	total: 31.3s	remaining: 6.59s
826:	learn: 0.5826528	total: 31.3s	remaining: 6.55s
827:	learn: 0.5826048	total: 31.4s	remaining: 6.52s
828:	learn: 0.582505

969:	learn: 0.5759953	total: 36.7s	remaining: 1.14s
970:	learn: 0.5759522	total: 36.8s	remaining: 1.1s
971:	learn: 0.5759163	total: 36.8s	remaining: 1.06s
972:	learn: 0.5758845	total: 36.8s	remaining: 1.02s
973:	learn: 0.5758332	total: 36.9s	remaining: 984ms
974:	learn: 0.5758062	total: 36.9s	remaining: 946ms
975:	learn: 0.5757186	total: 36.9s	remaining: 908ms
976:	learn: 0.5756713	total: 37s	remaining: 871ms
977:	learn: 0.5756368	total: 37s	remaining: 833ms
978:	learn: 0.5755664	total: 37.1s	remaining: 795ms
979:	learn: 0.5755294	total: 37.1s	remaining: 757ms
980:	learn: 0.5754827	total: 37.1s	remaining: 719ms
981:	learn: 0.5754125	total: 37.2s	remaining: 681ms
982:	learn: 0.5753865	total: 37.2s	remaining: 643ms
983:	learn: 0.5753482	total: 37.2s	remaining: 606ms
984:	learn: 0.5753228	total: 37.3s	remaining: 568ms
985:	learn: 0.5752292	total: 37.3s	remaining: 530ms
986:	learn: 0.5751605	total: 37.4s	remaining: 492ms
987:	learn: 0.5750643	total: 37.4s	remaining: 454ms
988:	learn: 0.575

In [115]:
test_values = pd.read_csv('encoded_test_values.csv').set_index('building_id')
test_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86868 entries, 300051 to 501372
Data columns (total 61 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   geo_level_1_id                          86868 non-null  int64 
 1   geo_level_2_id                          86868 non-null  int64 
 2   geo_level_3_id                          86868 non-null  int64 
 3   count_floors_pre_eq                     86868 non-null  int64 
 4   age                                     86868 non-null  int64 
 5   area_percentage                         86868 non-null  int64 
 6   height_percentage                       86868 non-null  int64 
 7   land_surface_condition                  86868 non-null  object
 8   foundation_type                         86868 non-null  object
 9   roof_type                               86868 non-null  object
 10  ground_floor_type                       86868 non-null  object
 

In [116]:
test_values = test_values[
    ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other',
       'roof_type_C0', 'roof_type_C1', 'land_surface_condition_C1',
       'land_surface_condition_C0', 'foundation_type_C0', 'foundation_type_C1',
       'foundation_type_C2', 'ground_floor_type_C0', 'ground_floor_type_C1',
       'ground_floor_type_C2', 'other_floor_type_C0', 'other_floor_type_C1',
       'other_floor_type_C2', 'position_C0', 'position_C1', 'position_C2',
       'plan_configuration_C0', 'plan_configuration_C1',
       'plan_configuration_C2', 'plan_configuration_C3',
       'legal_ownership_status_C0', 'legal_ownership_status_C1',
       'legal_ownership_status_C2']
]

for col in test_values.columns:
    if col.startswith('has_superstructure') or col.startswith('has_secondary_use'):
        test_values[col] = test_values[col].astype(bool)

test_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86868 entries, 300051 to 501372
Data columns (total 53 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   geo_level_1_id                          86868 non-null  int64
 1   geo_level_2_id                          86868 non-null  int64
 2   geo_level_3_id                          86868 non-null  int64
 3   count_floors_pre_eq                     86868 non-null  int64
 4   age                                     86868 non-null  int64
 5   area_percentage                         86868 non-null  int64
 6   height_percentage                       86868 non-null  int64
 7   has_superstructure_adobe_mud            86868 non-null  bool 
 8   has_superstructure_mud_mortar_stone     86868 non-null  bool 
 9   has_superstructure_stone_flag           86868 non-null  bool 
 10  has_superstructure_cement_mortar_stone  86868 non-null  bool 
 11  has_super

In [127]:
prediction =eclf.predict(test_values)

In [128]:
sub_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [129]:
submission = pd.DataFrame(data=prediction, columns=sub_format.columns, index=sub_format.index)
submission.to_csv('submission_ensamble_3_modelos_sin_grid_search.csv')

In [130]:
submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3
